In [333]:
import numpy as np

In [343]:
def householder_reflection(x):
    norm_x = np.linalg.norm(x)
    v = x.copy()
    v[0] += np.sign(x[0]) * norm_x
    v = v / np.linalg.norm(v)
    H = np.eye(len(x)) - 2 * np.outer(v, v)
    return H

def RRQR(A, eps, verbose=False):
    m, n = A.shape
    if (m < n):
        print('error m < n')
        return 0, 0, 0
    norms = np.array([(np.linalg.norm(A[:, i]), i, -i) for i in range(n)], dtype = [('norm', float), ('index', int), ('anti_index', int)])
    norms = np.sort(norms, order=['norm', 'anti_index'])[::-1]
    perm = norms['index']
    A_perm = A[:, perm]
    inv_perm = np.zeros(n).astype(int)
    for i in range(n):
        inv_perm[perm[i]] = i
    if verbose:
        print('A\n', A, '\n')
        print('norms\n', norms, '\n')
        print('permutation\n', perm, '\n')
        print('inverse permutation\n', inv_perm, '\n')
        print('permuted A\n', A_perm, '\n')
        print('inverse permuted\n', (A_perm[:, inv_perm]), '\n')
    stop_iter = n
    Q = np.eye(m)
    R = A_perm.copy()
    for k in range(n):
        if verbose:
            print(k + 1, f'iter of {n}')
        x = R[k:, k]
        H_k = householder_reflection(x)
        H_k_full = np.eye(m)
        H_k_full[k:, k:] = H_k
        R = H_k_full @ R
        Q = Q @ H_k_full
        if (k + 1 < n) and np.abs(R[k+1,k+1]) < eps:
            print(f'stopping iterations with R[{k+2}, {k+2}] = {R[k+1,k+1]} < {eps} after {k + 1} of {n} iterations')
            stop_iter = k + 1
            break
    Q = Q[:,:stop_iter]
    R = np.triu(R[:stop_iter,:])    
    A_perm = A[:,perm]
    if verbose:
        print('A_perm\n', A_perm, '\n')
        print('Q\n', Q, '\n')
        print('R\n', R, '\n')
        print('diff\n', Q @ R - A_perm)
    print('rrqr_diff_norm =', np.linalg.norm(Q @ R - A_perm, ord='fro'))
    return Q, R, inv_perm

In [344]:
A = np.random.normal(loc=10, scale=5, size=(5,4))
Q, R, P = RRQR(A, 1e-8, True)
print('pqr diff =', np.linalg.norm((Q @ R)[:, P] - A, ord='fro'))
print('\nnp Q\n', np.linalg.qr(A)[0])
print('\nnp R\n', np.linalg.qr(A)[1])
print('\nnp diff\n', np.linalg.qr(A)[0] @ np.linalg.qr(A)[1] - A)
print('np_diff_norm =', np.linalg.norm(np.linalg.qr(A)[0] @ np.linalg.qr(A)[1] - A, ord='fro'), '\n')

A
 [[ 7.94530085 13.71231475 10.39437726 15.65830235]
 [10.63027936  5.84933758 10.02918395 16.14798911]
 [10.76288563 15.19173756 12.8149968   2.84097472]
 [ 9.88450479 16.11591989  7.4517219  13.17763934]
 [ 4.48706453 14.05621541  5.28020878 15.65177235]] 

norms
 [(30.53914338, 3, -3) (30.1716967 , 1, -1) (21.36025553, 2, -2)
 (20.24370359, 0,  0)] 

permutation
 [3 1 2 0] 

inverse permutation
 [3 1 2 0] 

permuted A
 [[15.65830235 13.71231475 10.39437726  7.94530085]
 [16.14798911  5.84933758 10.02918395 10.63027936]
 [ 2.84097472 15.19173756 12.8149968  10.76288563]
 [13.17763934 16.11591989  7.4517219   9.88450479]
 [15.65177235 14.05621541  5.28020878  4.48706453]] 

inverse permuted
 [[ 7.94530085 13.71231475 10.39437726 15.65830235]
 [10.63027936  5.84933758 10.02918395 16.14798911]
 [10.76288563 15.19173756 12.8149968   2.84097472]
 [ 9.88450479 16.11591989  7.4517219  13.17763934]
 [ 4.48706453 14.05621541  5.28020878 15.65177235]] 

1 iter of 4
2 iter of 4
3 iter of 4
4 i

In [345]:
A = np.random.normal(loc=10, scale=5, size=(100,50))
Q, R, P = RRQR(A, 1e-8, False)
print('pqr diff =', np.linalg.norm((Q @ R)[:, P] - A, ord='fro'))
print('np_diff_norm =', np.linalg.norm(np.linalg.qr(A)[0] @ np.linalg.qr(A)[1] - A, ord='fro'), '\n')

rrqr_diff_norm = 8.574660387764163e-13
pqr diff = 8.574660387764163e-13
np_diff_norm = 2.752454386489642e-13 



In [346]:
n = 100
A = np.zeros((n, n))
for i in range(n - 1):
    A[i, i + 1] = 1.0
Q, R, P = RRQR(A, 1e-8, False)
print('pqr diff =', np.linalg.norm((Q @ R)[:, P] - A, ord='fro'))
print('np_diff_norm =', np.linalg.norm(np.linalg.qr(A)[0] @ np.linalg.qr(A)[1] - A, ord='fro'), '\n')

stopping iterations with R[100, 100] = 0.0 < 1e-08 after 99 of 100 iterations
rrqr_diff_norm = 0.0
pqr diff = 0.0
np_diff_norm = 0.0 



In [347]:
n = 1000
A = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        A[i, j] = np.sin(i + j)
Q, R, P = RRQR(A, 1e-8, False)
print('pqr diff =', np.linalg.norm((Q @ R)[:, P] - A, ord='fro'))
print('np_diff_norm =', np.linalg.norm(np.linalg.qr(A)[0] @ np.linalg.qr(A)[1] - A, ord='fro'), '\n')

stopping iterations with R[3, 3] = 6.493511274746708e-16 < 1e-08 after 2 of 1000 iterations
rrqr_diff_norm = 7.42554805918766e-09
pqr diff = 7.425548059187657e-09
np_diff_norm = 3.382197859067353e-13 



In [348]:
n = 5000
A = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        A[i, j] = np.sin(i + j)
Q, R, P = RRQR(A, 1e-8, False)
print('pqr diff =', np.linalg.norm((Q @ R)[:, P] - A, ord='fro'))
print('np_diff_norm =', np.linalg.norm(np.linalg.qr(A)[0] @ np.linalg.qr(A)[1] - A, ord='fro'), '\n')

stopping iterations with R[3, 3] = 9.461808007083764e-18 < 1e-08 after 2 of 5000 iterations
rrqr_diff_norm = 3.3863133697387156e-08
pqr diff = 3.386313369738998e-08
np_diff_norm = 5.831489138106291e-12 

